In [1]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import foundation as fd
import foundation.util as util
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision
import h5py as hf
from itertools import product

import ZODB, ZODB.FileStorage
import ZEO
#import neo
import transaction
import persistent
from persistent.list import PersistentList
from persistent.mapping import PersistentMapping
from BTrees import OIBTree, IOBTree, IIBTree
from html.parser import HTMLParser

from zodb_wikigraph import create_database, WikiGraph_DB
from extraction import wiki_expand, download_article_html

In [2]:
seeds = ['Artificial_intelligence', 'Computational_physics', 'Control_theory', 'Machine_learning', 'Quantum_computing']

seed = seeds[0]
#seed = 'Zero-knowledge_proof'
#seed = 'Cryptography'

In [40]:
links, redirs, _ = wiki_expand(seed)
links, redirs = [[link['href'][6:] for link in group] for group in [links, redirs]]
links, redirs = map(list, map(set, [links, redirs]))
if seed in links:
    links.remove(seed)
len(links), len(redirs)

** Bad header might be missing: Catastrophic_interference#The_Sequential_Learning_Problem
** Bad header might be missing: Superintelligence
** Bad header might be missing: 2001
** Bad header might be missing: 2001
** Bad header might be missing: 2001
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Superintelligence


(1399, 186)

In [31]:
# html = download_article_html(seed)
# len(html)

In [32]:
# for link in links:
#     if 'Cryptography' in link:
#         print(link)

In [8]:
addr, stop = ZEO.server(path=db_path)
addr, stop

(('127.0.0.1', 64399),
 <function ZEO._forker.start_zeo_server.<locals>.stop(stop_timeout=99)>)

In [9]:
wiki = WikiGraph_DB(addr)

In [24]:
wiki.root

<root: ID_table article_list rID_table redir_list>

In [41]:
wiki.add(seed, links, redirs)

In [45]:
len(wiki.root.article_list), len(wiki.root.ID_table)

(2111, 2111)

In [43]:
wiki.root.article_list[-5:]

[Article[2106](Decision_network, n=1, l=[]),
 Article[2107](Glossary_of_botany, n=1, l=[]),
 Article[2108](Knowledge_representation, n=1, l=[]),
 Article[2109](Closed_world_assumption, n=1, l=[]),
 Article[2110](Intractably, n=1, l=[])]

In [46]:
#print([article.num for article in wiki.root.article_list])

In [22]:
q = mp.Queue()

In [24]:
q.put(2)

In [25]:
len(q)

TypeError: object of type 'Queue' has no len()

In [32]:
ZEO.client?

In [34]:
client = ZEO.client(addr, read_only=True)

In [44]:
client.blob_dir

In [45]:
from ZEO import ClientStorage

In [ ]:
storage = ClientStorage.ClientStorage(addr, read_only=True)
db = ZODB.DB(storage)
conn = db.open()
root = conn.root

In [3]:
db_path = 'beta.test'

In [4]:
create_database(db_path)

In [5]:
def init_worker(**other_args):
    
#     out = {
#         #'is_new_article' : lambda x: x not in todo and x not in completed,
#     }
    
    return out

def worker_expand(todo, completed, update_lock):
    
    if len(todo) == 0:
        return None
    
    print(len(todo))
    #print(todo['Artificial_intelligence'])
    
    update_lock.acquire()
    article_name = next(iter(todo.keys()))
    completed[article_name] = 1
    del todo[article_name]
    update_lock.release()
    
    print(article_name)
    
    links, redirs, _ = wiki_expand(article_name)
    groups = [[link['href'][6:] for link in group] for group in [links, redirs]]
    links, redirs = map(list, map(set, groups))
    if seed in links:
        links.remove(seed)
        
    todo.update({link:1 for link in links if (link not in todo and link not in completed)})
    
    return article_name, links, redirs
        
    #len(links), len(redirs)

In [6]:
addr, stop = ZEO.server(path=db_path)
addr, stop

(('127.0.0.1', 49793),
 <function ZEO._forker.start_zeo_server.<locals>.stop(stop_timeout=99)>)

In [7]:
num_workers = 4

In [8]:
wiki = WikiGraph_DB(addr)

In [9]:
manager = mp.Manager()
todo = manager.dict()
completed = manager.dict()

In [10]:
shared_args = {}
private_args = {
    'todo': todo,
    'completed': completed,
    'update_lock': mp.Lock(),
}

In [11]:
todo.update({name:1 for name in seeds})
len(todo), list(todo.keys())

(5,
 ['Artificial_intelligence',
  'Computational_physics',
  'Control_theory',
  'Machine_learning',
  'Quantum_computing'])

In [12]:
creator = util.Farmer(fn=worker_expand, private_args=private_args, init_fn=None, 
                      num_workers=num_workers, waiting=10, auto_dispatch=True)

5
5
5
Artificial_intelligence
Computational_physics
Control_theory
2
Machine_learning
299
Quantum_computing
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Artificial_Intelligence
597
Computational_complexity_theory
973
Yukawa_potential
** Bad header might be missing: Catastrophic_interference#The_Sequential_Learning_Problem
** Bad header might be missing: Superintelligence
** Bad header might be missing: 2001
** Bad header might be missing: 2001
** Bad header might be missing: 2001
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Artificial_Intelligence
** Bad header might be missing: Superintelligence
993
Stark_effect
1161
Mathematical_physics
1847
Heliophysics
2894
Parallel_computing
2893
Chaos_theory
3163
Ordinary_differential_equation
** Bad header might be missing: Chaos
** Bad header might be missing: Chaos
** Bad header might be missing: Chaos
3

In [14]:
itr = 0
print_freq = 1
for out in creator:
    if out is None:
        continue
    
    #print('Out {}'.format(out[0]))
    
    wiki.add(*out)
    
    if itr % print_freq == 0:
        print('Itr {}K: todo={}, completed={}, db={}'.format(itr//1000 + 1, len(todo), len(completed), len(wiki)))
    
    if itr == 10:
        break
    
    itr += 1

Itr 1K: todo=4702, completed=22, db=3864
Itr 1K: todo=4926, completed=23, db=4015
Itr 1K: todo=5076, completed=24, db=4348
Itr 1K: todo=5075, completed=25, db=4671
Itr 1K: todo=5324, completed=26, db=4909
Itr 1K: todo=5374, completed=27, db=5105
Itr 1K: todo=5666, completed=28, db=5201
Itr 1K: todo=5782, completed=29, db=5393
Itr 1K: todo=5816, completed=30, db=5458
Itr 1K: todo=5827, completed=31, db=5583
Itr 1K: todo=5950, completed=32, db=5668


In [16]:
a = wiki.get_article(1,2,3,4)
len(a)

4

In [15]:
a = set()

In [19]:
wiki.get_ID('Spin_model')

4

In [16]:
a.add(1)

In [17]:
a

{1}